In [424]:
from bs4 import BeautifulSoup as bs
import urllib2
import re
import csv
from collections import OrderedDict

In [425]:
fandom = 'http://archiveofourown.org/tags/Final%20Fantasy%20VII/works'

In [485]:
def find_page(base_url, page_number):
    #go to any page number.
    return base_url+'?page=' +str(page_number)

In [486]:
def find_works(page):
    #Find all works from a works list page.
    works_page = bs(urllib2.urlopen(page))
    links = []
    for link in works_page.find_all('a'):
        url = link.get('href')
        url_s = [i for i in url.split('/') if i != '']
        if 'work' in url and len(url_s) == 2 and str(url_s[1]).isdigit():
                links.append('http://archiveofourown.org'+link.get('href'))
    return links

In [529]:
def show_allch(url):
    base = bs(urllib2.urlopen(url))
    full_url = url
    for link in base.find_all('a'):
        if 'Entire Work' in link.text:
            full_url = 'http://archiveofourown.org' + link.get('href')
    return full_url

In [532]:
def get_contents(url):
    #get work metadata and contents from the work page.
    print 'Reading url:', url
#     try:
    page = bs(urllib2.urlopen(url))
    content = str(page.body.text.encode('utf-8'))
#     contents = re.findall('Work Header+.+Actions',text,flags=re.DOTALL)[0].replace('\n','A')
#     except:
#         print 'Unable to read from this url'
#         contents = ''
#         with open('/Users/jingy/Desktop/problematic_url.csv', 'a') as g:
#             g.write(url)
    return url, contents

In [502]:
c = get_contents('http://archiveofourown.org/works/5205566')
f = [i for i in re.findall('Fandoms:A          AAA(.*?)AAAA\
    |Fandom:A          AAA(.*?)AAAA',str(c))[0] if i != ''][0]
f

Reading url: http://archiveofourown.org/works/5205566


"Bobobo-bo Bo-bobo (Anime & Manga)Soul Eater (Anime)Kingdom HeartsCave StoryPokemonHaloBlazBlue\\xe6\\x96\\xb0\\xe3\\x83\\xbb\\xe5\\x85\\x89\\xe7\\xa5\\x9e\\xe8\\xa9\\xb1 \\xe3\\x83\\x91\\xe3\\x83\\xab\\xe3\\x83\\x86\\xe3\\x83\\x8a\\xe3\\x81\\xae\\xe9\\x8f\\xa1 | Kid Icarus: Uprising (Video Game)RWBYMaximum Ride - James PattersonDanny PhantomRockman Zero | Mega Man ZeroSubarashiki Kono Sekai | The World Ends With YouLaw of TalosAssassin\\'s CreedFinal Fantasy VIIVocaloidPowerpuff GirlsPrototype (Video Games)HellsingKirby (Video Games)HomestuckDragon BallSonic the Hedgehog (Video Games)Kuroshitsuji | Black ButlerGrim Tales (Webcomic)"

In [490]:
def write_header():
    f = open('/Users/jingy/Desktop/works.csv', 'a')
    writer = csv.writer(f, delimiter=',')
    keys = ['Additional Tags', 'Archive Warning', 'Author', 'Category', 'Characters', 'Fandoms', 'Language', 'Notes',\
            'Rating', 'Relationship', 'Stats', 'Summary', 'Text', 'Title']
    writer.writerow(keys)
    f.close()

In [491]:
def write_work_content(work_dict):
    #write work metadata and contents as values of a sorted dictionary.
    f = open('/Users/jingy/Desktop/works.csv', 'a')
    writer = csv.writer(f, delimiter=',')
#     try:
    writer.writerow(OrderedDict(sorted(work_dict.items())).values())
#     OrderedDict(sorted(d.items())).values()
#     except:
#         pass
    f.close()

In [533]:
#creates dictionary for information in a single work.
def create_work_dict(url, contents):
    #get work metadata and contents into a dictrionary.
    print 'Getting work information from:', url
#     try:
    work = {}
    work['Rating'] = str(re.findall('Not Rated|General Audiences|Teen And Up Audiences|Mature|Explicit',contents)[0]).strip()
    work['Archive Warning'] = re.findall('Creator Chose Not To Use Archive Warnings|Graphic Depictions Of Violence\
    |Major Character Death|No Archive Warnings Apply|Rape/Non-Con|Underage',contents)[0].strip()
    work['Category'] = str(re.findall('F/F|F/M|Gen|M/M|Multi|Other',contents)[0])
    work['Fandoms'] = [i for i in re.findall('Fandoms:A          AAA(.*?)AAAA\
    |Fandom:A          AAA(.*?)AAAA',contents)[0] if i != ''][0]
    relationship = re.findall('Relationship:A          AAA(.*?)AAAA',contents)
    if relationship == []:
        work['Relationship'] = ''
    else:
        work['Relationship'] = relationship[0]
    work['Characters'] = re.findall('Characters:A          AAA(.*?)AAAA',contents)[0]
    work['Additional Tags'] = re.findall('Additional Tags:A          AAA(.*?)AAAA',contents)[0].strip()
    work['Language'] =  re.findall('Language:A      AA(.*?)A      A',contents)[0].strip()
    work['Stats'] = re.findall('Stats:AAA(.*?)AAAAAAAA',contents)[0].strip()
    work['Author'] = re.findall('A    AA(.*?)AAA',contents)[0].strip()
    work['Text']= [i for i in re.findall('Work Text:(.*?)AAAAAAAA|Chapter TextA(.*?)AAAAA',contents)[0] if i != ''][0].strip()
    work['Title']  = re.findall('AAAAAAAA      (.*?)A    AA',contents)[0].strip()
    work['Summary'] = re.findall('Summary:AAA(.*?)AAAAA',contents)[0].strip()
    work['Notes'] = re.findall('Notes:AA(.*?)AA',contents)[0].strip()
    print 'Finished with:', url
#     except:
#         print 'Something went wrong.'
    return work

In [503]:
d = create_work_dict('http://archiveofourown.org/works/5205566',str(c))

Getting work information from: http://archiveofourown.org/works/5205566
Finished with: http://archiveofourown.org/works/5205566


In [505]:
worklist

['http://archiveofourown.org/works/4214751',
 'http://archiveofourown.org/works/3966586',
 'http://archiveofourown.org/works/5205566',
 'http://archiveofourown.org/works/4373063',
 'http://archiveofourown.org/works/4255134',
 'http://archiveofourown.org/works/5202845',
 'http://archiveofourown.org/works/5059657',
 'http://archiveofourown.org/works/5044171',
 'http://archiveofourown.org/works/3426482',
 'http://archiveofourown.org/works/1543778',
 'http://archiveofourown.org/works/5198324',
 'http://archiveofourown.org/works/5191499',
 'http://archiveofourown.org/works/3344000',
 'http://archiveofourown.org/works/4747361',
 'http://archiveofourown.org/works/5186363',
 'http://archiveofourown.org/works/5185049',
 'http://archiveofourown.org/works/5184905',
 'http://archiveofourown.org/works/5184620',
 'http://archiveofourown.org/works/5184485',
 'http://archiveofourown.org/works/3855223']

In [537]:
page1 = find_page(fandom, 1)
worklist = find_works(page1)
write_header()
for i in worklist:
    f = show_allch(i)
    print 'full url:', f
    u, c = get_contents(u)
    print url
    work = create_work_dict(u, c)
#     print work
    write_work_content(work)

full url: http://archiveofourown.org/works/4214751?view_full_work=true
Reading url: http://archiveofourown.org/works/3855223
http://archiveofourown.org/works/3855223
Getting work information from: http://archiveofourown.org/works/3855223
Finished with: http://archiveofourown.org/works/3855223
full url: http://archiveofourown.org/works/3966586?view_full_work=true
Reading url: http://archiveofourown.org/works/3855223
http://archiveofourown.org/works/3855223
Getting work information from: http://archiveofourown.org/works/3855223
Finished with: http://archiveofourown.org/works/3855223
full url: http://archiveofourown.org/works/5205566
Reading url: http://archiveofourown.org/works/3855223
http://archiveofourown.org/works/3855223
Getting work information from: http://archiveofourown.org/works/3855223
Finished with: http://archiveofourown.org/works/3855223
full url: http://archiveofourown.org/works/4373063?view_full_work=true
Reading url: http://archiveofourown.org/works/3855223
http://archiv

In [ ]:
base = bs(urllib2.urlopen('http://archiveofourown.org/works/4373063/chapters/9925442'))

In [ ]:
if 'page' in link.get('href'):
        links.append(link.get('href'))
next_url = 'http://archiveofourown.org/'+links[-1]
next_page = bs(urllib2.urlopen(next_url))
return next_url

In [384]:
# import re
# from itertools import izip

# # s = 'Title: Example: of a title Date: November 23 1234 Other: Other information'

# keys = ['Rating', 'Archive Warning', 'Other']
# pattern = re.compile('({})\s+'.format(':|'.join(keys)))

# print dict(izip(*[(i.strip() for i in (pattern.split(contents)) if i)]*2))

{}


In [351]:
# def find_nextpage(base):
#     links = []
#     for link in base.find_all('a'):
#         if 'page' in link.get('href'):
#             links.append(link.get('href'))
#     next_url = 'http://archiveofourown.org/'+links[-1]
#     next_page = bs(urllib2.urlopen(next_url))
#     return next_url